In [2]:
import pyspark
import urllib.request
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('widikidiw') \
    .getOrCreate()

In [2]:
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-02.parquet"
filename = "fhvhv_tripdata_2021-02.parquet"

urllib.request.urlretrieve(url, filename)

('fhvhv_tripdata_2021-02.parquet', <http.client.HTTPMessage at 0x24590474e10>)

In [6]:
df = spark.read.parquet('fhvhv_tripdata_2021-02.parquet')
df.show(20)

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [8]:
df.createOrReplaceTempView("dfliat")

In [10]:
#1. How many taxi trips were there on February 15?
q1 = spark.sql("SELECT count(*) Trip_Count FROM dfliat Where cast(dropoff_datetime as date) = '2021-02-15' ")
q1.show()

+----------+
|Trip_Count|
+----------+
|    427850|
+----------+



In [11]:
#2. Find the longest trip for each day?
q2 = spark.sql("SELECT cast(dropoff_datetime as date) Trip_Day, max(trip_miles) as Longest_Trip_Miles \
                      FROM dfliat \
                      where cast(dropoff_datetime as date) is not null \
                      group by cast(dropoff_datetime as date) \
                      order by cast(dropoff_datetime as date) asc ")
q2.show()

+----------+------------------+
|  Trip_Day|Longest_Trip_Miles|
+----------+------------------+
|2021-02-01|            212.43|
|2021-02-02|            282.78|
|2021-02-03|            161.12|
|2021-02-04|            203.97|
|2021-02-05|            239.16|
|2021-02-06|            275.32|
|2021-02-07|            266.36|
|2021-02-08|             253.5|
|2021-02-09|            219.43|
|2021-02-10|             512.5|
|2021-02-11|            417.77|
|2021-02-12|            250.11|
|2021-02-13|            233.44|
|2021-02-14|            207.44|
|2021-02-15|            184.77|
|2021-02-16|           307.661|
|2021-02-17|            324.19|
|2021-02-18|            527.11|
|2021-02-19|           201.743|
|2021-02-20|            329.16|
+----------+------------------+
only showing top 20 rows



In [13]:
#3. Find Top 5 Most frequent `dispatching_base_num`?
q3 = spark.sql("SELECT dispatching_base_num, count(*) as Trip_Day \
                      FROM dfliat \
                      group by dispatching_base_num \
                      order by count(*) desc limit 5 ")
q3.show()


+--------------------+--------+
|dispatching_base_num|Trip_Day|
+--------------------+--------+
|              B02510| 3233664|
|              B02764|  965568|
|              B02872|  882689|
|              B02875|  685390|
|              B02765|  559768|
+--------------------+--------+



In [14]:
#4. Find Top 5 Most common location pairs (PUlocationID and DOlocationID)?
q4 = spark.sql("SELECT PUlocationID, DOlocationID, count(*) as Trip_Day \
                      FROM dfliat \
                      group by PUlocationID, DOlocationID \
                      order by count(*) desc limit 5 ")
q4.show()

+------------+------------+--------+
|PUlocationID|DOlocationID|Trip_Day|
+------------+------------+--------+
|          76|          76|   45041|
|          26|          26|   37329|
|          39|          39|   28026|
|          61|          61|   25976|
|          14|          14|   17934|
+------------+------------+--------+

